In [53]:
import cv2
import os

def load_images_from_folder(folder):
    X_train = []
    Y_label=[]
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.resize(img, (32, 32),cv2.INTER_LINEAR)
        if img is not None:
            X_train.append(img)
            Y_label.append(filename[0])
    return [np.array(X_train,dtype=np.float64),np.array(Y_label)]


In [101]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.compat.v1 as tf
import keras

tf.disable_v2_behavior()

katalog_train="C:/Users/dom/ZDJ1"
katalog_test="C:/Users/dom/ZDJ2"

(X_train, Y_train_label), (X_test, Y_test) = keras.datasets.mnist.load_data()
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))

Y_train_label =  np.array(Y_train_label)
unikalne_klasy = np.unique(Y_train_label)

print("unikalne_klasy = {}".format(unikalne_klasy))
liczba_klas = len(unikalne_klasy)
wys=28
szer=28
num_skladowych_kolorow=1

# Y_train = np.eye(liczba_klas).astype(np.int)[(Y_train_label == unikalne_klasy[1]).astype(np.int)]

Y_train = np.array([Y_train_label == unikalne_klasy[i] for i in range(liczba_klas)]).astype(np.int).T

x_pl = tf.placeholder(tf.float32, shape=[None, wys, szer, num_skladowych_kolorow], name='Input')
y_pl = tf.placeholder(tf.float32, shape=[None, liczba_klas], name='Output')


### Zdefiniuj warstwy ukryte convulsional i pool ###

conv1 = tf.layers.conv2d(
    inputs=x_pl,
    filters=32,
    kernel_size=[3, 3],
    padding='SAME',
    activation=tf.nn.relu
)
pool1 = tf.layers.max_pooling2d(conv1, pool_size=[2, 2], strides=2, padding='VALID')

conv2 = tf.layers.conv2d(
    inputs=pool1,
    filters=64,
    kernel_size=[3, 3],
    padding='SAME',
    activation=tf.nn.relu
)
pool2 = tf.layers.max_pooling2d(conv2, pool_size=[2, 2], strides=2, padding='VALID')


conv3 = tf.layers.conv2d(
    inputs=pool2,
    filters=64,
    kernel_size=[3, 3],
    padding='SAME',
    activation=tf.nn.relu
)
pool3 = tf.layers.max_pooling2d(conv3, pool_size=[2, 2], strides=2, padding='VALID')


# flat = tf.reshape(pool3, [-1, 4 * 4 * 64])
flat = tf.layers.flatten(pool3)

fc1 = tf.layers.dense(inputs=flat, units=100, activation=tf.nn.relu)
fc2 = tf.layers.dense(inputs=fc1, units=10)
softmax = tf.nn.softmax(fc2)
################################################################################


      
correct_prediction = tf.equal(tf.argmax(y_pl, axis=1), tf.argmax(softmax, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))



loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_pl, logits=fc2))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(loss)

num_epoch = 150
batch_size = np.minimum(1000, X_train.shape[0])
print("batch_size = {}".format(batch_size))


nr = np.arange(X_train.shape[0])
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    # conv1_, pool1_, conv2_, pool2_, conv3_, pool3_, flat_, fc1_ = sess.run([conv1, pool1, conv2, pool2, conv3, pool3, flat, fc1], feed_dict={x_pl: X_train})
    for i in range(num_epoch):
        np.random.shuffle(nr)
        nr_used_train = nr[:batch_size]
        nr_used_acc = nr[-batch_size:]
        accuracy_1, _, loss_ = sess.run([accuracy, train, loss], feed_dict= {x_pl : X_train[nr_used_train], y_pl: Y_train[nr_used_train]})
        accuracy_2 = sess.run(accuracy,
                                      feed_dict={x_pl: X_train[nr_used_acc, :, :, :], y_pl: Y_train[nr_used_acc, :]})
        print("epoka {}, loss = {}, acc = {}, ({})".format(i, loss_, accuracy_2, accuracy_1))

    Y_test_est_cont = sess.run(softmax, feed_dict = {x_pl : X_test})
    Y_test_est_disc = np.argmax(Y_test_est_cont, axis = 1)
    print(Y_test_est_cont)
    print(Y_test_est_disc)

n_zgodnosc = 1
acc = 0
print(X_test.shape[0])



unikalne_klasy = [0 1 2 3 4 5 6 7 8 9]


C:\Users\dom\AppData\Local\Temp\ipykernel_12436\3220342688.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Y_train = np.array([Y_train_label == unikalne_klasy[i] for i in range(liczba_klas)]).astype(np.int).T
C:\Users\dom\AppData\Local\Temp\ipykernel_12436\3220342688.py:34: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  conv1 = tf.layers.conv2d(
C:\Users\dom\AppData\Local\Temp\ipykernel_12436\3220342688.py:41: UserWarning: `tf.layers.max_pooling2d` is depr

batch_size = 1000
epoka 0, loss = 41.900943756103516, acc = 0.08900000154972076, (0.10599999874830246)
epoka 1, loss = 41.340309143066406, acc = 0.08100000023841858, (0.09700000286102295)
epoka 2, loss = 29.017255783081055, acc = 0.22699999809265137, (0.0860000029206276)
epoka 3, loss = 13.918793678283691, acc = 0.2029999941587448, (0.22300000488758087)
epoka 4, loss = 16.484027862548828, acc = 0.25099998712539673, (0.1850000023841858)
epoka 5, loss = 12.006089210510254, acc = 0.2619999945163727, (0.21799999475479126)
epoka 6, loss = 8.338155746459961, acc = 0.2630000114440918, (0.2930000126361847)
epoka 7, loss = 7.6395649909973145, acc = 0.25999999046325684, (0.24500000476837158)
epoka 8, loss = 6.098692417144775, acc = 0.3230000138282776, (0.25200000405311584)
epoka 9, loss = 4.668527126312256, acc = 0.35899999737739563, (0.2849999964237213)
epoka 10, loss = 4.0486016273498535, acc = 0.3499999940395355, (0.33399999141693115)
epoka 11, loss = 3.632614850997925, acc = 0.34299999475479

epoka 98, loss = 0.2079363763332367, acc = 0.9620000123977661, (0.9459999799728394)
epoka 99, loss = 0.18198320269584656, acc = 0.9520000219345093, (0.9459999799728394)
epoka 100, loss = 0.19601351022720337, acc = 0.953000009059906, (0.9369999766349792)
epoka 101, loss = 0.1644691824913025, acc = 0.9509999752044678, (0.949999988079071)
epoka 102, loss = 0.18218642473220825, acc = 0.9480000138282776, (0.9459999799728394)
epoka 103, loss = 0.2002101093530655, acc = 0.9419999718666077, (0.9449999928474426)
epoka 104, loss = 0.17814426124095917, acc = 0.9589999914169312, (0.9440000057220459)
epoka 105, loss = 0.17170602083206177, acc = 0.9649999737739563, (0.949999988079071)
epoka 106, loss = 0.1626829206943512, acc = 0.9419999718666077, (0.949999988079071)
epoka 107, loss = 0.16864298284053802, acc = 0.9490000009536743, (0.953000009059906)
epoka 108, loss = 0.1366608887910843, acc = 0.9509999752044678, (0.953000009059906)
epoka 109, loss = 0.1437271386384964, acc = 0.9509999752044678, (0.

In [102]:
for i in range(X_test.shape[0]):
    if Y_test_est_disc[i] == Y_test[i]:
       acc_flag = 1
    else:
        acc_flag = 0
    acc += acc_flag
    print("plik = {} zaklasyfikowany jako {}, ({}), if_OK = {}".format(Y_test[i], Y_test_est_disc[i], Y_test_est_cont[i], acc_flag))

acc = acc/X_test.shape[0]
# print(Y_test_est_disc)
print(X_train.shape)
print("ACC = {}".format(acc))

plik = 7 zaklasyfikowany jako 7, ([6.8881519e-07 5.0642091e-09 2.2427387e-06 2.9588639e-07 5.4288063e-12
 1.4472559e-08 2.7279903e-08 9.9999678e-01 2.1274269e-09 3.9800661e-08]), if_OK = 1
plik = 2 zaklasyfikowany jako 2, ([1.1573173e-06 2.7555916e-05 9.9989223e-01 7.7756425e-07 3.2107823e-07
 3.8328139e-06 6.9928385e-05 4.4165060e-07 3.4865627e-06 1.5536891e-07]), if_OK = 1
plik = 1 zaklasyfikowany jako 1, ([2.9826853e-05 9.9918562e-01 5.9427998e-06 1.0605472e-06 2.8535683e-04
 2.7759771e-07 3.6637677e-04 1.2521436e-04 3.5545165e-07 3.2285861e-08]), if_OK = 1
plik = 0 zaklasyfikowany jako 0, ([9.9958986e-01 5.4415434e-08 1.5422796e-04 1.6917090e-06 5.5672058e-07
 1.1045009e-06 2.1041217e-04 6.9691864e-06 3.3649223e-05 1.5313417e-06]), if_OK = 1
plik = 4 zaklasyfikowany jako 4, ([5.6071787e-07 4.3716273e-08 2.2180207e-06 5.6362870e-10 9.9961263e-01
 6.3849420e-08 4.1979209e-07 7.7114407e-08 6.0096639e-08 3.8386695e-04]), if_OK = 1
plik = 1 zaklasyfikowany jako 1, ([5.2267892e-06 9.9889

 1.2061896e-07 1.3182228e-04 2.9443344e-04 3.4181468e-04 4.2437287e-06]), if_OK = 1
plik = 3 zaklasyfikowany jako 3, ([6.7938288e-07 1.3614404e-05 3.6675399e-04 9.3270779e-01 6.1726217e-07
 9.8474966e-03 7.0956317e-05 1.4375920e-05 2.9593304e-02 2.7384486e-02]), if_OK = 1
plik = 1 zaklasyfikowany jako 1, ([9.8514893e-06 9.9768460e-01 9.9883218e-06 1.7490230e-06 1.7323985e-03
 1.2815355e-05 4.2119995e-04 3.3180928e-05 9.2123635e-05 2.0237271e-06]), if_OK = 1
plik = 5 zaklasyfikowany jako 5, ([1.51112312e-09 9.54859047e-07 6.36520170e-08 2.56053340e-06
 1.16687985e-12 9.99981642e-01 1.35461836e-10 3.02156633e-10
 1.18872819e-07 1.46235325e-05]), if_OK = 1
plik = 1 zaklasyfikowany jako 1, ([3.9136544e-06 9.9914432e-01 8.4799751e-05 3.6620742e-04 1.4555872e-04
 1.0005110e-05 1.4706202e-04 1.0317126e-05 6.7622561e-05 2.0179563e-05]), if_OK = 1
plik = 2 zaklasyfikowany jako 2, ([7.1263639e-06 1.9734034e-06 9.9996650e-01 1.6271824e-05 2.3154576e-09
 6.7055468e-09 7.8065996e-06 6.9377933e-08 2

 2.6150797e-06 1.6920589e-04 1.8009356e-05 1.5008220e-06 5.2805003e-08]), if_OK = 1
plik = 8 zaklasyfikowany jako 8, ([4.4773151e-05 5.9450108e-06 1.4490455e-05 5.6280731e-03 4.1263091e-05
 1.0283393e-04 2.1278454e-05 5.1963277e-04 9.9233413e-01 1.2876628e-03]), if_OK = 1
plik = 8 zaklasyfikowany jako 8, ([5.6968853e-02 1.6516689e-05 9.0386393e-03 1.2477445e-02 1.2707898e-03
 6.7968917e-04 4.1382754e-04 1.4223885e-04 9.1037643e-01 8.6156586e-03]), if_OK = 1
plik = 2 zaklasyfikowany jako 2, ([1.3130860e-07 5.7455338e-07 9.9982637e-01 1.7241329e-04 2.8491317e-11
 1.5382929e-08 4.2098810e-07 1.7829608e-08 2.4025457e-09 3.4609460e-09]), if_OK = 1
plik = 3 zaklasyfikowany jako 3, ([1.0349952e-07 7.3819997e-06 9.6760759e-06 9.9973434e-01 1.5929258e-10
 2.1323909e-05 9.0216696e-09 3.3327415e-05 1.5175075e-05 1.7859145e-04]), if_OK = 1
plik = 6 zaklasyfikowany jako 6, ([1.9889025e-02 1.1850763e-03 4.8676037e-04 9.1830743e-06 7.0427173e-05
 1.5836349e-01 8.1744188e-01 1.0293479e-06 2.5270092e-0

 2.60303728e-04 9.25214708e-01]), if_OK = 1
plik = 8 zaklasyfikowany jako 8, ([3.9746947e-06 1.8508699e-05 7.7634543e-04 1.0484222e-02 1.9994954e-05
 5.3493993e-04 1.8773044e-05 1.1691719e-02 9.3325925e-01 4.3192275e-02]), if_OK = 1
plik = 8 zaklasyfikowany jako 2, ([7.4295845e-04 2.5558896e-07 8.9142907e-01 3.5230499e-03 1.3014243e-05
 4.8360285e-05 5.8221645e-05 4.2610985e-04 1.0276223e-01 9.9666405e-04]), if_OK = 0
plik = 6 zaklasyfikowany jako 6, ([1.20824398e-05 1.53084807e-06 2.78510997e-05 2.43948193e-07
 9.66613129e-07 9.96905001e-05 9.99723256e-01 7.20768867e-07
 1.02332844e-04 3.14291356e-05]), if_OK = 1
plik = 0 zaklasyfikowany jako 0, ([9.9910980e-01 5.4556267e-08 4.3649536e-05 2.1465567e-05 8.4286021e-06
 4.0343070e-06 2.8614655e-05 9.7082630e-05 8.0542341e-06 6.7870325e-04]), if_OK = 1
plik = 0 zaklasyfikowany jako 0, ([9.9943429e-01 1.3084535e-10 5.0165643e-05 1.8140692e-05 3.7748563e-07
 1.6051007e-07 1.0088566e-05 4.5201642e-04 2.6890788e-05 7.7619225e-06]), if_OK = 1


plik = 3 zaklasyfikowany jako 3, ([3.4493155e-09 4.6538178e-05 6.2227141e-06 9.9951017e-01 1.3560679e-10
 4.0883454e-04 8.6373475e-08 2.6629359e-06 8.7150347e-06 1.6794984e-05]), if_OK = 1
plik = 8 zaklasyfikowany jako 8, ([5.0150102e-04 6.9581908e-08 1.2699278e-01 1.0532936e-02 9.0485952e-07
 4.1994613e-04 7.7637116e-04 9.0534950e-04 8.5881037e-01 1.0598006e-03]), if_OK = 1
plik = 6 zaklasyfikowany jako 6, ([6.3249824e-04 8.8848591e-02 8.6816566e-05 2.6237114e-08 1.1615973e-03
 1.5184345e-02 8.9243644e-01 1.1181192e-05 1.6270851e-03 1.1330506e-05]), if_OK = 1
plik = 4 zaklasyfikowany jako 4, ([8.8115698e-08 2.8730272e-05 1.1421470e-08 2.1071738e-09 9.9994886e-01
 1.8207450e-07 5.8139422e-06 1.7094196e-06 1.4275335e-07 1.4421380e-05]), if_OK = 1
plik = 2 zaklasyfikowany jako 2, ([1.1555120e-05 1.9826922e-07 9.9823582e-01 1.6979679e-03 2.1448821e-08
 8.7277868e-08 2.3241315e-05 2.6138650e-06 2.8459079e-05 5.4567522e-08]), if_OK = 1
plik = 0 zaklasyfikowany jako 0, ([9.9881387e-01 4.1319

...

 9.9965072e-01 2.2165866e-08 1.4309333e-09 8.1528788e-06 3.3730312e-04]), if_OK = 1
plik = 6 zaklasyfikowany jako 6, ([1.1752544e-02 2.4239771e-05 8.9377172e-05 1.6492218e-05 1.2558716e-04
 9.2672699e-06 9.8612577e-01 2.2937381e-04 1.5452207e-03 8.2061779e-05]), if_OK = 1
plik = 4 zaklasyfikowany jako 4, ([9.7626880e-06 1.8193077e-04 1.7662728e-07 1.0452113e-09 9.9968886e-01
 2.0393530e-08 3.9285635e-05 7.6198383e-05 1.3221209e-08 3.7659304e-06]), if_OK = 1
plik = 3 zaklasyfikowany jako 3, ([5.5083667e-09 1.4685806e-07 1.5603648e-05 9.9992621e-01 1.2228282e-12
 4.4612032e-05 4.7897448e-08 1.3948063e-06 1.1849614e-05 1.3349293e-07]), if_OK = 1
plik = 0 zaklasyfikowany jako 0, ([9.9938548e-01 8.8070081e-08 5.9151909e-05 6.5155712e-07 4.8085479e-07
 1.5164194e-09 5.1374064e-04 3.9352894e-05 9.8460748e-07 2.6476172e-09]), if_OK = 1
plik = 0 zaklasyfikowany jako 0, ([9.9965703e-01 1.1247207e-06 1.4140051e-04 1.2681414e-05 7.3338242e-07
 4.4979838e-09 7.4136311e-05 1.0135301e-04 1.1557311e-0

 2.9978618e-02 9.4963300e-01 6.1347387e-06 1.6924348e-02 1.7927941e-04]), if_OK = 1
plik = 6 zaklasyfikowany jako 6, ([2.34080479e-03 8.43105256e-04 3.78415716e-05 5.49748984e-05
 5.43465503e-05 2.28626393e-02 9.68519866e-01 5.30841453e-06
 5.26896305e-03 1.21556595e-05]), if_OK = 1
plik = 3 zaklasyfikowany jako 3, ([9.5015857e-08 6.1758732e-07 1.8009671e-07 9.9881232e-01 5.9995854e-12
 1.1388804e-03 1.4417383e-09 1.9072473e-06 2.0177004e-06 4.4081116e-05]), if_OK = 1
plik = 2 zaklasyfikowany jako 2, ([7.0929091e-06 2.7899205e-04 8.5871583e-01 6.3816257e-02 1.6562933e-04
 2.1657730e-02 1.8431791e-04 5.3153988e-02 2.8195221e-04 1.7381915e-03]), if_OK = 1
plik = 6 zaklasyfikowany jako 6, ([2.3249297e-05 2.3305872e-04 1.3035552e-02 1.5752768e-04 5.9600847e-05
 7.9732694e-02 9.0106326e-01 4.1521539e-06 5.6165764e-03 7.4416341e-05]), if_OK = 1
plik = 3 zaklasyfikowany jako 3, ([1.21518555e-08 2.85394407e-07 1.93254255e-06 9.99975562e-01
 1.94127365e-14 1.81902342e-05 1.34846756e-09 1.463269

 7.6108091e-02 8.7181064e-08 6.3395946e-06 8.7591223e-05 2.5423747e-04]), if_OK = 1
plik = 4 zaklasyfikowany jako 4, ([1.9419383e-06 8.5236968e-07 4.6414897e-07 4.6860955e-09 9.9999142e-01
 3.7079968e-09 7.7385425e-07 3.1341699e-06 2.6854406e-07 1.2016649e-06]), if_OK = 1
plik = 7 zaklasyfikowany jako 7, ([6.09530252e-05 1.27037183e-07 1.00181652e-02 9.06628964e-04
 1.00145385e-07 8.72101100e-06 4.13989073e-06 9.87261653e-01
 9.05569526e-04 8.33994418e-04]), if_OK = 1
plik = 2 zaklasyfikowany jako 2, ([5.6506778e-06 1.7311565e-07 9.9920911e-01 4.8652393e-05 5.6483064e-08
 2.5198796e-07 4.6994028e-05 6.5451377e-04 3.1957326e-05 2.5972881e-06]), if_OK = 1
plik = 0 zaklasyfikowany jako 0, ([9.9196702e-01 5.2276244e-10 5.5208011e-03 3.8636821e-07 1.6568318e-06
 6.5722794e-08 2.4487195e-03 1.8384144e-06 5.6178149e-05 3.3566382e-06]), if_OK = 1
plik = 5 zaklasyfikowany jako 5, ([3.7326890e-06 1.0988188e-05 1.5083803e-06 8.0458369e-05 1.3243498e-08
 9.9986839e-01 2.2567140e-07 1.2517665e-06 7

 5.7484321e-08 5.5700293e-05 3.6782389e-08 8.5561061e-07 1.9160465e-08]), if_OK = 1
plik = 2 zaklasyfikowany jako 2, ([3.2507680e-05 7.7769604e-09 9.9784672e-01 2.0495807e-03 1.7176177e-09
 1.5228060e-06 9.2866384e-07 1.6416577e-07 6.8111498e-05 5.1665404e-07]), if_OK = 1
plik = 4 zaklasyfikowany jako 4, ([1.5127016e-04 7.0291600e-04 6.5802828e-06 3.5168529e-07 9.9755472e-01
 9.2084048e-07 2.0126468e-05 7.8164297e-04 4.7110220e-07 7.8097306e-04]), if_OK = 1
plik = 7 zaklasyfikowany jako 7, ([7.7163196e-07 1.8312640e-08 1.5725072e-06 6.3623503e-07 9.7053485e-09
 6.5436275e-08 2.6976545e-09 9.9998426e-01 4.5569859e-09 1.2606679e-05]), if_OK = 1
plik = 8 zaklasyfikowany jako 8, ([3.0666026e-06 5.7272808e-07 7.1165668e-06 3.1635263e-05 1.6898341e-05
 1.7673185e-05 2.3748903e-06 6.2037889e-06 9.9948454e-01 4.2990263e-04]), if_OK = 1
plik = 0 zaklasyfikowany jako 0, ([9.9998105e-01 5.9223852e-09 5.4254356e-06 5.2571205e-09 1.4572498e-11
 1.2713361e-06 1.1878844e-05 9.6280095e-09 5.7962854e-0

 9.9994540e-01 1.0765474e-09 1.9366639e-07 4.1287039e-06 3.4838984e-05]), if_OK = 1
plik = 0 zaklasyfikowany jako 0, ([9.9827874e-01 1.0834941e-04 4.5059607e-04 3.2974015e-06 1.6837504e-07
 1.2144706e-05 1.0879604e-03 4.1414783e-05 1.5020325e-05 2.4030596e-06]), if_OK = 1
plik = 7 zaklasyfikowany jako 7, ([1.2966660e-05 1.2740545e-06 5.1186289e-06 2.8801405e-06 7.8106950e-08
 2.7003800e-08 3.2277325e-07 9.9997127e-01 2.3379517e-08 6.0593875e-06]), if_OK = 1
plik = 2 zaklasyfikowany jako 2, ([2.1212520e-06 1.6518531e-08 9.9992001e-01 2.8568928e-05 4.1513284e-10
 1.2069903e-07 3.0456569e-05 6.6339753e-06 1.1801689e-05 2.1842244e-07]), if_OK = 1
plik = 3 zaklasyfikowany jako 3, ([6.2561659e-05 1.5185097e-03 9.8524690e-03 9.8687243e-01 8.2845322e-09
 1.6776822e-03 4.5908073e-06 8.2390784e-07 1.0321586e-05 6.6884490e-07]), if_OK = 1
plik = 6 zaklasyfikowany jako 6, ([1.2104807e-06 2.6437527e-07 7.1655886e-06 1.7908754e-07 2.1794722e-06
 9.0957656e-06 9.9997675e-01 3.6921168e-07 2.0407908e-0